@takamichitoda is already doing something similar approach with pytorch.
https://www.kaggle.com/takamichitoda/ventilator-train-classification

In my latest version (Larger Epoch size and add some features), LB score was 0.186..

In [ ]:
import os
import pickle
import random
import numpy as np
import pandas as pd
import seaborn as sns

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.callbacks import CSVLogger, ModelCheckpoint
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import LearningRateScheduler
from tensorflow.keras.optimizers.schedules import ExponentialDecay

from sklearn.metrics import mean_absolute_error as mae
from sklearn.preprocessing import RobustScaler, normalize
from sklearn.model_selection import train_test_split, GroupKFold, KFold

from IPython.display import display

In [ ]:
from tqdm import tqdm

In [ ]:
def seed_everything(seed=0):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    tf.random.set_seed(seed)

seed_everything(0)

In [ ]:
path_folder_out = "/kaggle/working"
print(f"output folder is {path_folder_out}")
# os.makedirs(path_folder_out, exist_ok=True)

In [ ]:
DEBUG = False

# In kaggle
path_folder = "../input/ventilator-pressure-prediction"

train_ori = pd.read_csv(f'{path_folder}/train.csv')
test_ori = pd.read_csv(f'{path_folder}/test.csv')
submission = pd.read_csv(f'{path_folder}/sample_submission.csv')

if DEBUG:
    train_ori = train_ori[:80*1000]

In [ ]:
def add_features(df):
    df['cross'] = df['u_in']*df['u_out']
    df['cross2'] = df['time_step']*df['u_out']
    
    df['u_in_cumsum'] = (df['u_in']).groupby(df['breath_id']).cumsum()
    df['one'] = 1
    df['count'] = (df['one']).groupby(df['breath_id']).cumsum()
    df['u_in_cummean'] = df['u_in_cumsum'] / df['count']
    
    # LagFeatures
    df['u_in_lag1'] = df.groupby('breath_id')['u_in'].shift(1)
    df['u_in_lag2'] = df.groupby('breath_id')['u_in'].shift(2)
    df['u_in_lag3'] = df.groupby('breath_id')['u_in'].shift(3)
    df['u_in_lag4'] = df.groupby('breath_id')['u_in'].shift(4)
    df = df.fillna(0)
    
    # diff values
    df['u_in_max'] = df.groupby(['breath_id'])['u_in'].transform('max')
    df['u_in_diff1'] = df['u_in'] - df['u_in_lag1']
    df['u_in_diff2'] = df['u_in'] - df['u_in_lag2']
    df['u_in_diffmax'] = df.groupby(['breath_id'])['u_in'].transform('max') - df['u_in']
    df['u_in_diffmean'] = df.groupby(['breath_id'])['u_in'].transform('mean') - df['u_in']

    # AUC
    df['breath_time_lag1'] = df.groupby("breath_id")["time_step"].shift(1).fillna(0)
    df['breath_time_lag2'] = df.groupby("breath_id")["time_step"].shift(2).fillna(0)
    df["area"] = df["breath_time_lag1"] * (df["u_in"] + df["u_in_lag1"]) / 2.0
    df["AUC"] = df.groupby("breath_id")["area"].cumsum()

    # R,C
    df['R'] = df['R'].astype(str)
    df['C'] = df['C'].astype(str)
    df['RC'] = df['R']+df['C']
    df = pd.get_dummies(df)

    return df

train = add_features(train_ori)
test = add_features(test_ori)

In [ ]:
def simple_check_df(df):
    print("np.inf", df.isin([np.inf, -np.inf]).any().any())
    print("np.nan", df.isnull().any().any())
simple_check_df(train)
simple_check_df(test)

In [ ]:
d1 = {}
d2 = {}
for i, v in enumerate(sorted(set(train["pressure"]))):
    d1[v] = i
    d2[i] = v

In [ ]:
targets = train[['pressure']].replace(d1).astype(int).to_numpy().reshape(-1, 80)
dropcols = [
  'id',
  'breath_id',
  'one',
  'count',
]

train.drop(dropcols, axis=1, inplace=True)
test.drop(dropcols, axis=1, inplace=True)
train.drop("pressure", axis=1, inplace=True)

In [ ]:
test2 = test.head(1600)

In [ ]:
RS = RobustScaler()
train = RS.fit_transform(train)
test = RS.transform(test)
test2 = RS.transform(test2)

train = train.reshape(-1, 80, train.shape[-1])
test = test.reshape(-1, 80, train.shape[-1])
test2 = test2.reshape(-1, 80, train.shape[-1])

In [ ]:
EPOCH = 3 # only for debug
BATCH_SIZE = 256

kf = KFold(n_splits=5, shuffle=True, random_state=2021)
test_preds = []
for fold, (train_idx, test_idx) in enumerate(kf.split(train, targets)):
    if fold != 0: # only for debug
        continue
    
    print('-'*15, '>', f'Fold {fold+1}', '<', '-'*15)
    X_train, X_valid = train[train_idx], train[test_idx]
    y_train, y_valid = targets[train_idx], targets[test_idx]
    model = keras.models.Sequential([
        keras.layers.Input(shape=train.shape[-2:]),
        keras.layers.Bidirectional(keras.layers.LSTM(300, return_sequences=True)),
        keras.layers.Bidirectional(keras.layers.LSTM(250, return_sequences=True)),
        keras.layers.Bidirectional(keras.layers.LSTM(150, return_sequences=True)),
        keras.layers.Bidirectional(keras.layers.LSTM(100, return_sequences=True)),
        keras.layers.Dense(50, activation='selu'),
        keras.layers.Dense(950, activation='softmax'),
    ])
    model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics='accuracy')

    # 学習
    cp = ModelCheckpoint(f"model.keras_{fold}.best_weight.h5", monitor="val_loss", verbose=1, save_best_only=True, save_weights_only=False)
    scheduler = ExponentialDecay(1e-3, 400*((len(train)*0.8)/BATCH_SIZE), 1e-5)
    lr = LearningRateScheduler(scheduler, verbose=1)
    es = EarlyStopping(monitor="val_loss", patience=100, verbose=1, mode="min", restore_best_weights=True)
    csv_logger = CSVLogger(f'training.{fold}.log', append=True)
    model.fit(X_train, y_train, validation_data=(X_valid, y_valid), epochs=EPOCH, batch_size=BATCH_SIZE, callbacks=[lr, csv_logger, es, cp])

In [ ]:
def model_split_predict(num_split=500):
    # global var => model, test
    preds_flatten = np.array([])
    tests_splitted = np.array_split(test, 500)
    for test_splitted in tqdm(tests_splitted):
        test_preds = model.predict(test_splitted)
        for i in range(len(test_splitted)):
            preds_flatten = np.append(preds_flatten, np.argmax(test_preds[i], axis=1))
    return preds_flatten

In [ ]:
test_preds_index = model_split_predict()

In [ ]:
test_preds = [d2[ind] for ind in test_preds_index]